In [28]:
!pip install fasttext
!pip install praw

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━

In [5]:
from numpy import dot
from numpy.linalg import norm
import praw


In [67]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English

'cc.en.300.bin'

In [7]:
ft = fasttext.load_model('cc.en.300.bin')

## Testing FastText

In [8]:
def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))

In [9]:
sports_embedding = ft.get_word_vector("sports")
soccer_embedding = ft.get_word_vector("soccer")
random = ft.get_word_vector("random")

In [10]:
cosine_similarity(sports_embedding, soccer_embedding)

0.59049165

In [11]:
cosine_similarity(random, soccer_embedding)

0.07309229

In [12]:
a = ft.get_word_vector("soccer")
b = ft.get_word_vector("manchesterunited")
cosine_similarity(a,b)

0.26300964

## Scraping Reddit usernames
Assumption that 0.2 similarity threshold indicates the username is relevant to the subreddit

In [31]:
#limit 30 requests per minute
reddit = praw.Reddit(client_id="jAaMNZwg_71Q1xD3fgqKMA",
                      client_secret="OQ199_gL5-k_Fem5TDh51g0P5lI7Og",
                      user_agent="code_broken",
                      check_for_async=False)

In [62]:
# subreddits
sports = reddit.subreddit("sports")
gaming = reddit.subreddit("gaming")
music = reddit.subreddit("music")

In [87]:
def get_usernames_for_subreddit(subreddit, subreddit_name):
    """Extracts usernames from a subreddit with a high similarity score
    
    Uses fasttext embedding to calculat similarity between the author's name
    and the subreddit name. Note that praw automatically ratelimits and batches
    the response from subreddit.top. max limit for each request is 1000
    
    """
    subreddit_embedding = ft.get_word_vector(subreddit_name)
    usernames = set()
    for i, post in enumerate(subreddit.top(time_filter="year", limit=1000)):
        print(i, end='\r')
        if post.author is not None:
            author_embedding = ft.get_word_vector(post.author.name)
            distance = cosine_similarity(author_embedding, subreddit_embedding)
            if distance > 0.2:
                usernames.add((author, distance))
                print((author, distance))
            
            # check comments as well. Extremely time consuming to run...
#             post.comments.replace_more(limit=None)
#             for j, comment in enumerate(post.comments):
#                 print(i, j, end='\r')
#                 if comment.author is not None:
#                     comment_author = comment.author.name
#                     if distance > 0.2:
#                         usernames.add((comment_author, distance))
#                         print((comment_author, distance))
    return usernames

In [88]:
get_usernames_for_subreddit(sports, "sports")

('jisooya1432', 0.21210432)
('jisooya1432', 0.21210432)


{('jisooya1432', 0.21210432)}

In [70]:
# trying to do extraction on all posts
# problem is subreddit names are often hard to understand
data = set()
popular = reddit.subreddit("popular")
for i, comment in enumerate(popular.comments(limit=10000)):
    print(i, end='\r')
    if comment.author is not None:
        author_embedding = ft.get_word_vector(comment.author.name)
        subreddit_embedding = ft.get_word_vector(comment.subreddit.display_name)
        distance = cosine_similarity(author_embedding, subreddit_embedding)
        if distance > 0.2:
            data.add((comment.subreddit.display_name, comment.author.name, distance))
            print((comment.subreddit.display_name, comment.author.name, distance))


('amIuglyBrutallyHonest', 'Kavooch', 0.43664742)
('foodhacks', 'OPsMomHuffsFartJars', 0.22890162)
('tooktoomuch', 'pinksparklecat', 0.3267806)
('u_Mxrcuriie21', 'AdoptMeTigers', 0.22194073)
('shitposting', 'AutoModerator', 0.33177036)
('totalwar', 'JNHaddix', 0.32495505)
('Stormlight_Archive', 'not__get', 0.28860295)
('adnansyed', 'strmomlyn', 0.20456232)
('xxfitness', '2PoopOrNot2Poop', 0.377956)
('pokemongo', 'AutoModerator', 0.31289122)
('avatartrading', 'blockchainsniper', 0.4341562)
('u_Left_Egg_1626', 'Stryger85', 0.2858205)
('StardewValley', 'Vinyl_DjPon3', 0.3813513)
('TrueOffMyChest', 'maycontainduck', 0.36519668)
('Bossfight', 'Alien_X10', 0.37715876)
('totallynotrobots', 'screechingahhhhhh', 0.35343722)
('drugcirclejerk', 'sloppyasseating', 0.26447985)
('WhitePeopleTwitter', 'Senior-Conversation8', 0.24895656)
('justneckbeardthings', 'SnooTangerines8313', 0.38725883)
('snapchatspamgroups', 'TommyEs17', 0.23653203)
('residentevil', 'CrimsonPants', 0.28146887)
('Breadit', 'jjv

## Using the public bigquery dataset
There's a public bigquery dataset called fh-bigquery.reddit_posts that contains many posts from around 2015. Let's use that data instead

In [41]:
import pandas as pd
import tqdm

In [42]:
def extract_relevant_usernames_from_dataset(dataset_prefix, threshold = 0.2):
    """Use fasttext to extract usernames who's similarity score exceeds the threshold
    
    Input is a dataset prefix name, they are stored locally as {dataset_prefix}_usernames.csv
    Similarity score is calculated as cosine similarity
    """
    df = pd.read_csv(dataset_prefix + "_usernames.csv")
    print(dataset_prefix, len(df))
    subreddit_embedding = ft.get_word_vector(dataset_prefix)
    
    data = set()
    for index, row in tqdm.tqdm(df.iterrows()):
        author_embedding = ft.get_word_vector(row.author)
        distance = cosine_similarity(subreddit_embedding, author_embedding)
        if distance > threshold:
            data.add((dataset_prefix, row.author, distance))
    return data

In [43]:
datasets = ["sports", "funny", "gaming", "music", "soccer", "science", "food", "art"]
data = set()
for dataset in datasets:
    data.update(extract_relevant_usernames_from_dataset(dataset, threshold=0.15))

sports 11759


11759it [00:01, 11445.04it/s]


funny 307163


307163it [00:27, 11090.48it/s]


gaming 145922


...

22789it [00:02, 11054.30it/s]


food 27917


27917it [00:02, 10867.96it/s]


art 13495


13495it [00:01, 10889.94it/s]


In [44]:
df = pd.DataFrame(data, columns=['category', 'username', 'score'])
df.to_csv('categorized_usernames.csv')
print(len(df))
print(df.groupby(['category']).count())
print(df.sample(n=20))

23110
          username  score
category                 
art            498    498
food           636    636
funny        14933  14933
gaming        4122   4122
music         1312   1312
science        765    765
soccer         695    695
sports         149    149
      category             username     score
3464     funny            rodneyb74  0.161907
9618     music          grickrizzle  0.157236
2713     funny          cb_sunshine  0.160711
7033     funny          madamemoody  0.170248
6623     funny            Jackass44  0.154933
9056     funny           beezybwing  0.179670
9259     funny           likeamanda  0.153878
20939    funny             surprizd  0.150145
7026     funny             Snoepixx  0.169458
9554     funny  theyjustcallmeallie  0.155675
22474   gaming           cool_arr0w  0.166237
9039     funny   Posing_Hipster_011  0.180369
18459    funny      FlockofSeagulls  0.152463
14366    funny      HelpMeLoseMyFat  0.155237
2812     funny    Cheeseburgerjesus  0.15062

## Format for gpt2 and convert to text file

In [57]:
df = pd.read_csv('categorized_usernames.csv')
len(df)

23110

In [121]:
count = 0
for index, row in df.iterrows():
    if row.score > 0.2:
        count += 1
print(count)

3028


In [123]:
df.sample(n=50)

,Unnamed: 0,category,username,score
18086,18086,funny,AshTheBestiest,0.187893
17732,17732,funny,iHateReddit_srsly,0.177464
2942,2942,gaming,zombiezelda,0.153113
21592,21592,gaming,NinjaBear77,0.173753
20974,20974,funny,Twatbagel,0.151355
2214,2214,funny,indiefilmy,0.201931
10749,10749,funny,fartsarefunny27,0.219440
17300,17300,funny,NukeItGood,0.159891
7704,7704,funny,awesometheawessome,0.168161
22770,22770,gaming,shadow_Creeper,0.150286


In [58]:
text = ""
for index, row in df.iterrows():
    text += row.category + " " + row.username + "\n"
with open('gpt2_usernames.txt', 'w') as f:
    f.write(text)

In [59]:
text = ""
for index, row in df.iterrows():
    if row.score > 0.2:
        text += row.category + " " + row.username + "\n"
with open('gpt2_usernames_high_score.txt', 'w') as f:
    f.write(text)

In [36]:
!pip install -q gpt-2-simple

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://g

In [69]:
import tensorflow as tf
import gpt_2_simple as gpt2

In [73]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 634Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.90Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 676Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:28, 5.65Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 487Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.18Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 4.64Mit/s]                                                      


In [74]:
#tf.compat.v1.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess, dataset='gpt2_usernames_high_score.txt', model_name='124M', steps=1000, learning_rate=1e-5,
              restore_from='fresh', run_name='run1', print_every=10, sample_every=100x, save_every=200)

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.91it/s]

dataset has 19381 tokens
Training...


[10 | 297.93] loss=4.25 avg=4.25
[20 | 604.70] loss=4.15 avg=4.20
[30 | 989.00] loss=4.08 avg=4.16
[40 | 1428.81] loss=3.97 avg=4.11
[50 | 1863.89] loss=3.97 avg=4.08
[60 | 2306.57] loss=3.92 avg=4.05
[70 | 2746.76] loss=3.78 avg=4.01
[80 | 3143.25] loss=3.74 avg=3.98
[90 | 3542.00] loss=3.60 avg=3.94
[100 | 3927.08] loss=3.61 avg=3.90
[110 | 4286.03] loss=3.57 avg=3.87
[120 | 4641.08] loss=3.51 avg=3.84
[130 | 5007.17] loss=3.39 avg=3.80
[140 | 5387.21] loss=3.27 avg=3.76
[150 | 5745.73] loss=3.29 avg=3.73
[160 | 6072.50] loss=3.16 avg=3.69
[170 | 6385.34] loss=3.09 avg=3.65
[180 | 6678.61] loss=3.02 avg=3.61
[190 | 6966.84] loss=2.86 avg=3.57
[200 | 7255.34] loss=2.81 avg=3.53
======== SAMPLE 1 ========
awk, i-the-funny
funny TheJadedGirlfriend
funny jedimarky
art TheDriftalMcFlurry
funny dankerchief89
gaming Geek_Gambler
gaming MrZazzl
soccer futbologazionale
gaming CzaristC
gaming TheHeroinNinjas
funny funnyamerican
gaming NerdFaun
gaming TheNerdGamer
funny funnygifs
gaming TheGrea

In [75]:
gpt2.generate(sess, run_name='run1')

Doscodamus
funny DaWonderWaffle
gaming Nightstalker427
gaming rabidgamer
gaming anonymous
funny MildlyFunnyFunny
funny mikeyplacey
funny ohbodyside
soccer Cite Your Senses
science majorCDq
funny TomDoesHeCool
funny funnyiner8
funny Fartles-and-Booey
soccer Internazionale
gaming gamerdrome
music kendostickball
funny Howyyy_ (KPKDizzle)
funny sofaks
funny simplesfeld
funny simplesiconsimal
funny sombre
funny Wee_Moo
funny simplesesberry
music TinyHats
funny fwoO2aySiswhipe
gaming modMpWern
funny wildoomcow7
funny wildoomcow7susgnoll
funny wildoomcow7
art rbarker3
sports TonyC
science militaryForwardTeardrop
funny sillymr
gaming Ricer262
funny dumbasamazing
gaming ZeroRational
gaming Thebareknuckle
soccer soccerfreakbitch
funny firstdiev
funny MrFunnyShoes
gaming LxRuFx
funny DXingxxt
funny Random2dan
funny firstdiev99
funny zokh
funny Newcomer
science geekmom
funny Newcomer
gaming Zeoxady
funny first dfw
gaming Zeoxady
funny Newbie
funny theresek
funnystefanon
funny TheNewHero
funny Give

In [117]:
sess = gpt2.reset_session(sess)
gpt2.load_gpt2(sess,
              checkpoint='latest',
              run_name="run1")

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


In [120]:
gpt2.generate(sess, run_name='run1', prefix='basketball', length=100)

basketball 25kbroker BLOCKCASE GAMES 25kbroker XBLA
gaming blagobloggo
gaming lynxmoonfucker
soccer soccer_man
gaming XBLA
gaming rfa_fan
gaming edamame28
gaming iXenocider X
funny johnnychameleon
food tempastidz
funny Daniella_
funny brizzlenug
funny Pastafarian
funny ram


In [92]:
dataset_usernames = set(df['username'])

In [109]:
generations = ["rabidgamer", "gamer_panda", "geekmom", "discomfortable", "MrSmoob", "sombreep", "d3ngen6",
              "cbustersMeme", "Thetechdian", "futbologazionale"]
for g in generations:
    print(g, g in dataset_usernames)

rabidgamer False
gamer_panda False
geekmom False
discomfortable True
MrSmoob False
sombreep False
d3ngen6 False
cbustersMeme False
Thetechdian False
futbologazionale False


In [81]:
gpt2.download_gpt2(model_dir="models1", model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 801Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 1.76Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 868Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [02:58, 2.79Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 651Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.89Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 4.22Mit/s]                                                      


In [87]:
sess1 = gpt2.reset_session(sess1)

In [88]:
#tf.compat.v1.reset_default_graph()
sess1 = gpt2.start_tf_sess()
gpt2.finetune(sess1, dataset='gpt2_usernames.txt', model_name='124M', steps=1000, learning_rate=1e-5,
              restore_from='fresh', run_name='run2', print_every=10, sample_every=200, save_every=200,
              model_dir="models1", checkpoint_dir="checkpoint1")

Loading checkpoint models1/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models1/124M/model.ckpt
Loading dataset...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.43s/it]


dataset has 154721 tokens
Training...
[10 | 276.79] loss=4.37 avg=4.37
[20 | 565.97] loss=4.29 avg=4.33
[30 | 866.97] loss=4.27 avg=4.31
[40 | 1149.38] loss=4.27 avg=4.30
[50 | 1436.56] loss=4.12 avg=4.26
[60 | 1714.45] loss=4.19 avg=4.25
[70 | 1993.60] loss=4.16 avg=4.24
[80 | 2281.95] loss=4.16 avg=4.23
[90 | 2556.53] loss=4.15 avg=4.22
[100 | 2833.56] loss=4.10 avg=4.20
[110 | 3115.68] loss=4.07 avg=4.19
[120 | 3389.26] loss=4.06 avg=4.18
[130 | 3663.56] loss=4.07 avg=4.17
[140 | 3939.16] loss=4.03 avg=4.16
[150 | 4213.43] loss=4.01 avg=4.15
[160 | 4488.17] loss=4.06 avg=4.14
[170 | 4762.47] loss=4.07 avg=4.14
[180 | 5038.78] loss=4.04 avg=4.13
[190 | 5318.18] loss=4.01 avg=4.13
[200 | 5594.25] loss=3.98 avg=4.12
Saving checkpoint1/run2/model-200
INFO:tensorflow:checkpoint1/run2/model-200.data-00000-of-00001
INFO:tensorflow:497800
INFO:tensorflow:checkpoint1/run2/model-200.meta
INFO:tensorflow:501300
INFO:tensorflow:checkpoint1/run2/model-200.index
INFO:tensorflow:501300
======== SA

In [91]:
gpt2.generate(sess1, run_name='run2', model_dir="models1", checkpoint_dir="checkpoint1", prefix="funny")

funny Raffaelec7
gaming toothbun
funny ThisIsTheAwkwardFuture
funny angry_xoxo
funny alex_mclovin
funny Shitty_Goes
funny nikko
funny trenley
funny imdaddytime
funny luigwen
funny zephyr
funny lolwut11
funny xtasyafrf
funny njhaha
funny scumbagblister
funny lolwuthere
funny melancholy_vibrations
funny chadclovin
gaming the_academic_person
funny kiwizakart
gaming Master_Gamer
funny Ewok
funny wahrks
funny TheBigClanofScumbag
funny brittanyraco
funny the_great_dave
funny tamielgood321
funny frolicky88
funny heyjessie
funny mistermikey
funny skypezoo
funny fritzthebrit
funny lumpy
funny biker_lobster
funny snethers
gaming JMcGrowSucks
gaming darkknight3
funny flics
funny thechicfunky20s
gaming TheBadGamers
funny kengoa
funny shabanov
funny imforcleandlookedao
funny CptBubb_and_reddit
gaming thanikeb
gaming JonTheDragon
funny TheLlama
science 13371337
funny foo243
funny jktxx
funny shutup_dick
funny Bigboi18
funny Snorki_Snorki
funny FauxGirl
gaming chrispunditranks
funny thegreatnfl
funny